In [ ]:
# We won't need TensorFlow here
!pip uninstall -y tensorflow
# Install `transformers` from master
!pip install git+https://github.com/huggingface/transformers
!pip list | grep -E 'transformers|tokenizers'
# transformers version at notebook update --- 2.11.0
# tokenizers version at notebook update --- 0.8.0rc1

In [ ]:
%%time 
from pathlib import Path

from tokenizers import BertWordPieceTokenizer

#paths = [str(x) for x in Path(".").glob("**/*.txt")]
paths = r"/content/drive/MyDrive/100mbout/nout2006.txt"
# Initialize a tokenizer
tokenizer = BertWordPieceTokenizer()

# Customize training
tokenizer.train(files=paths, vocab_size=52_000, min_frequency=2,special_tokens=[
                "[UNK]",
                "[SEP]",
                "[PAD]",
                "[CLS]",
                "[MASK]",                                                               

              ])

CPU times: user 22min 11s, sys: 9.28 s, total: 22min 21s
Wall time: 11min 38s


In [ ]:
!mkdir bert
tokenizer.save_model("bert")

['bert/vocab.txt']

In [2]:
from tokenizers.implementations import BertWordPieceTokenizer
from tokenizers.processors import BertProcessing


tokenizer = BertWordPieceTokenizer(
    "./bert/vocab.txt",
    
)

In [3]:
tokenizer._tokenizer.post_processor = BertProcessing(
    ("[SEP]", tokenizer.token_to_id("[SEP]")),
    ("[CLS]", tokenizer.token_to_id("[CLS]")),
)
tokenizer.enable_truncation(max_length=512)

In [4]:
from transformers import BertConfig

config = BertConfig(
    vocab_size=52_000,
    max_position_embeddings=514,
    num_attention_heads=12,
    num_hidden_layers=6,
    type_vocab_size=1,
)

In [5]:
from transformers import BertTokenizerFast

tokenizer = BertTokenizerFast.from_pretrained("./bert", max_len=512)

In [6]:
from transformers import BertForMaskedLM

model = BertForMaskedLM(config=config)

In [7]:
model.num_parameters()

83504416

In [ ]:
!pip install datasets

In [ ]:
!pip install git+https://github.com/huggingface/transformers
!git clone https://github.com/huggingface/transformers.git
!pip list | grep -E 'transformers|tokenizers'

In [ ]:
!python /content/transformers/examples/pytorch/language-modeling/run_plm.py \
    --train_file "/content/drive/MyDrive/100mbout/200mb/nout2006.txt" \
    --tokenizer_name "/content/xlnet" \
    --do_train \
    --max_steps 1000 \
    --pad_to_max_length True \
    --output_dir /content/output_xlnet

In [ ]:
from transformers import LineByLineTextDataset

dataset = LineByLineTextDataset(
    tokenizer=tokenizer,
    file_path="/content/drive/MyDrive/100mbout/nout2006.txt",
    block_size=128,
)

/usr/local/lib/python3.7/dist-packages/transformers/data/datasets/language_modeling.py:124: FutureWarning: This dataset will be removed from the library soon, preprocessing should be handled with the 🤗 Datasets library. You can have a look at this example script for pointers: https://github.com/huggingface/transformers/blob/master/examples/pytorch/language-modeling/run_mlm.py
  FutureWarning,


In [ ]:
from transformers import DataCollatorForLanguageModeling

data_collator = DataCollatorForLanguageModeling(
    tokenizer=tokenizer, mlm=True, mlm_probability=0.15
)

In [ ]:
from transformers import Trainer, TrainingArguments

training_args = TrainingArguments(
    output_dir="./bert",
    max_steps = 1000,
    overwrite_output_dir=True,
    num_train_epochs=1,
    per_gpu_train_batch_size=64,
    save_steps=10_000,
    save_total_limit=2,
    prediction_loss_only=True,
)

trainer = Trainer(
    model=model,
    args=training_args,
    data_collator=data_collator,
    train_dataset=dataset,
)

In [ ]:
%%time
trainer.train()

In [ ]:
trainer.save_model("./bert")